# SVD Math

Let's explore the usage of SVD decomposition in numpy.

In [ ]:
import numpy as np
import seaborn as sb
from matplotlib import pyplot as plt
%matplotlib inline

### Singular Value Decomposition

#### SVD

If A is a $N \times n$ matrix, then the _Singular Value Decomposition_ (SVD) of $A$ is given by 

$$A = U \Sigma V^{T}$$

where

- $U$ and $V$ are orthogonal matrices, meaning that $UU^T = \textbf{1}$ and $VV^T = \textbf{1}$ are identity matrices, and
- $\Sigma$ is a diagonal matrix, meaning that the values lie *only* on the diagonal (This does not mean that $\Sigma$ is also a square matrix!)


- U shape: $N \times N$
- $\Sigma$ shape: $N \times n$
- VT shape: $n \times n$

In particular, $U$ and $V$ are square matrices consisting of the eigenvectors of $AA^T$ and $A^TA$, respectively. Note that $\Sigma$ is _not_ necessarily a square matrix (since generally $N \neq n$). Its values are non-negative real numbers, and are the _singular values_ of $A$, which are defined as the square root of the eigenvalues of $AA^T$ (or $A^TA$, which are the same).

Let's see that in action. Let's start with some arbitraty matrix $M$ and compute its SVD.

In [ ]:
M = np.random.random((5,3))
M

In [ ]:
U, s, VT = np.linalg.svd(M)  # returns matrix, array (only diagonal)
print U
print s
print VT

Note that `numpy`'s `linalg.svd` returns `U, s, VT`, where
- `s` is the actual diagonal of $\Sigma$ (not the entire matrix) and
- `VT` is the transpose of $V$.

Note that $\Sigma$ is a diagonal matrix, but not a square matrix.

In [ ]:
S = np.zeros(M.shape)
S[:min(M.shape), :min(M.shape)] = np.diag(s)
S

Let's check if the SVD equation holds:

In [ ]:
np.isclose(M, U.dot(S).dot(VT)).all()

Good!

Let's check if the values in $\Sigma$ are indeed the square roots of the eigenvalues of $AA^T$ resp. $A^TA$.  

We sort the eigenvalues, so we can compare them easily. We also round down to some decimal to clearly show the values that are nearly zero.

In [ ]:
values1 = np.sort(np.linalg.eig(M.dot(M.T))[0])[::-1].round(7)
values2 = np.sort(np.linalg.eig(M.T.dot(M))[0])[::-1].round(7)
squares = np.sort(np.square(s))[::-1].round(7)
print values1
print values2
print squares

Let's inspect $U$ check if its columns are indeed equal to the eigenvectors of $AA^T$.  

We sort the eigenvectors according to their eigenvalue, so we can compare them easily again.

In [ ]:
values, vectors = np.linalg.eig(M.dot(M.T))
vectors = vectors[:, values.argsort()[::-1]]
print U
print vectors
print U / vectors

Note that they're equal up to a possible sign (+/-), since the orientation of eigenvectors and eigenvalues is arbitrary. Also note that the last eigenvectors are just noise as their corresponding eigenvalues are near-zero.

Let's inspect $V$ and $A^TA$.

In [ ]:
V = VT.T
values, vectors = np.linalg.eig(M.T.dot(M))
vectors = vectors[:, values.argsort()[::-1]]
print V
print vectors
print V / vectors

Great, that all seems to work!

#### Truncated SVD

How does this decomposition help us with data science?  As we saw in the example above already, if $\Sigma$ is not a square matrix, a lot of the column vectors in $U$ are irrelevant since they correspond to zero-value eigenvalues. Moreover, in practice, you'd have many eigenvalues that are very small. Since the values in $\Sigma$ are ordered from big to small, we could significantly reduce the dimensions of $\Sigma$ by limiting this matrix to an actual square matrix of dimension $k$, only containing the first $k$ rows and columns of $\Sigma$, and throwing away the smallest singular values.

\begin{align*}
A = U \Sigma V^{T} \approx U' \Sigma' V'^{T} \\
\end{align*}

We often ommit these accents and simply write $A = U \Sigma V^{T}$ when we in fact mean the _truncated SVD_ decomposition. Since we truncate only the smallest singular values, this loss in accuracy is only small if we choose $k$ large enough. This is a small price for a huge gain in insight in our data.

In [ ]:
n_samples, n_features = M.shape
k = 1
M_svd = U[:n_samples, :k].dot(S[:k, :k]).dot(V[:n_features, :k].T)
print M
print M_svd 

How to measure the quality of this?

In [ ]:
s_kept = s[:k]
print "M is a %dx%d-matrix, but we only choose the largest %d singular values" % (n_samples, n_features, k)
print "\nWe kept %d%% of our singular values, weighted by their size:" % (100. * s_kept.sum() / s.sum())
print "- before:", s
print "- after: ", s_kept
diff = ((M_svd - M) / M).round(2)
print "\nRelative differences with original matrix (%.2f on average):" % diff.mean()
print diff

For an individual data point, the difference can be substantial, but for the population as a whole, we are pretty close to our initial matrix. 


#### Interpretation

Let's $A$ be our $N \times n$ feature matrix with $N$ samples and $n$ features, and let's write the SVD as follows, trunacted to some dimension $k < n$:

\begin{align}
A = U \Sigma V^{T}
\end{align}

Each of these matrices represents some grouping the data:
- $U$ ($N \times k$) represents how each datapoint relates to some latent concept,
- $V$ ($n \times k$) represents how each concept relates to each original feature, and
- $\Sigma$ ($k \times k$) represents the strength of each of those concepts in the dataset.

This reduces the number of dimensions from $n$ to $k$, since instead of modelling our data for each feature, we now model our data for each concept. Our intuition is that datapoints with similar relevant features will get similar concepts.

A very concrete example of this is text classification, in which case this method is often called _Latent semantic Analysis_ (LSA).  As always with Natural Language Processing (NLP), we transform our $N$ documents into a $N \times n$ feature matrix, where each column represents a certain word or n-gram (e.g., using `CountVectorizer`).  Please refer to the **notebook about LSA** in this folder for a clear demonstration about this.

### Principal Component Analysis

Principal Component Analysis (PCA) is directlty related to SVD by the following expression:

\begin{align*}
A &= U \Sigma V^{T} \\
AA^T &= (U \Sigma V^{T})(V \Sigma U^{T}) \\
&= U \Sigma^2 U^{T}
\end{align*}

Note that we used that $V$ is an orthogonal matrix, and hence $V^TV = \textbf{1}$.  Then we have:
- $AA^T$ is called the _covariance matrix_ (assuimg that all features $A$ were scaled with mean zero),
- $U$ is a square matrix consisting of the eigenvectors of $AA^T$, and
- $\Sigma^2$ is a square diagonal matrix with the eigenvalues of $AA^T$.

This means they are unique set of points that can be scaled by the covariance matrix but not rotated.  Therefore, they represent a new set of axes for our space.  This means our data can be shifted form our current axes or perspective to a new coordinate system - we can move our data points from samples in our feature space to samples in some underlying concept space. Pease refer to the **notebook about PCA** for an in-depth look into the math behind this.

### Further reading

- [Linear algebra explained in four pages](http://cnd.mcgill.ca/~ivan/miniref/linear_algebra_in_4_pages.pdf)
- [Latent Semantic Analysis](https://en.wikipedia.org/wiki/Latent_semantic_indexing) on Wikipedia